In [1]:
using JuMP
using Gurobi
using LinearAlgebra
using DataFrames, PrettyTables

# Problem data
c = [25.0, 50.0]                     # cost vector
w = 1.0                              # target return
ε = 0.05                             # violation probability
θ = 0.1                              # Wasserstein radius
x0 = 1.3                             # contextual feature realization

# mean and covariance
μ = [0.95, 0.88, 1.45]              # [X, Y1, Y2]
Σ = [ 1.0   0.20   0.30;
      0.20   0.50   0.25;
      0.30   0.25   0.80 ]           # full covariance

3×3 Matrix{Float64}:
 1.0  0.2   0.3
 0.2  0.5   0.25
 0.3  0.25  0.8

In [2]:
using Distributions, Random, Statistics
Random.seed!(1)
mv_normal = MvNormal(μ, Σ)
data = rand(mv_normal, 100)
μ̂ = vec(mean(data, dims=2))

dist = sqrt(norm(μ̂ - μ))

0.24787074199274664

In [3]:
# partition Σ and μ
μ̂X = μ̂[1]
μ̂Y = μ̂[2:3]
ΣXX = Σ[1, 1]
ΣXY = Σ[1, 2:3]
ΣYX = Σ[2:3, 1]
ΣYY = Σ[2:3, 2:3]

# conditional mean & covariance terms
A = ΣYX / ΣXX
ΣY_given_X = ΣYY .- ΣYX * ΣYX' ./ ΣXX
μY_given_X = μ[2:3] .+ ΣYX * inv(ΣXX) * (x0 - μ[1])
L = cholesky(Symmetric(ΣY_given_X)).L  # Cholesky factor
L2 = cholesky(Symmetric(ΣYY)).L  # Cholesky factor

Φinv = quantile(Normal(), ε)      # quantile of N(0,1), i.e. Φ^{-1}(ε)

-1.6448536269514729

In [4]:
cond_dist = MvNormal(μY_given_X, ΣY_given_X)
Random.seed!(2)
out_of_sample_data = rand(cond_dist, 10000)

2×10000 Matrix{Float64}:
 -0.00680161  1.35536  0.513005   0.797689  …  0.887506  0.136493  1.46637
  0.207381    1.62912  2.37213   -0.457829     2.28919   2.0735    0.607059

In [ ]:
results = DataFrame(
    epsilon = Float64[],
    theta = Float64[],
    opt_value = Any[],
    satisfaction = Any[],
    contextual_opt_value = Any[],
    contextual_satisfaction = Any[]
)

for ε in [0.05, 0.07, 0.09, 0.1]
    for θ in [0.01, 0.05, 0.1, 0.2]
        Φinv = quantile(Normal(), ε)      # quantile of N(0,1), i.e. Φ^{-1}(ε)

        ## Contextual Model
        model = Model(Gurobi.Optimizer)

        @variable(model, z[1:2] ≥ 0)
        @variable(model, t1 ≥ 0)
        @variable(model, t2 ≥ 0)

        # @constraint(model, sum(z) ≥ .1)
        # Robust chance constraint reformulation
        @constraint(model,
            dot(z, μ̂Y) + dot(z, A) * (x0 - μ̂X) - θ * t1 ≥ w - Φinv * t2
        )

        # SOC constraints
        @constraint(model, [t1, z[1], z[2]] in SecondOrderCone())
        @constraint(model, [t2; L * z] in SecondOrderCone())

        # Objective
        @objective(model, Min, dot(c, z))

        optimize!(model)
        if termination_status(model) == MOI.OPTIMAL
            contextual_opt_value = objective_value(model)

            # Out of sample satisfaction
            ẑ = value.(z)
            n_samples = size(out_of_sample_data, 2)
            count = 0

            for i in 1:n_samples
                if dot(out_of_sample_data[:, i], ẑ) ≥ w
                    count += 1
                end
            end

            contextual_satisfaction = count / n_samples
        else
            contextual_opt_value = nothing 
            contextual_satisfaction = nothing
        end

        ## Nominal Model
        model = Model(Gurobi.Optimizer)

        @variable(model, z[1:2] ≥ 0)
        @variable(model, t1 ≥ 0)
        @variable(model, t2 ≥ 0)

        # @constraint(model, sum(z) ≥ .1)
        # Robust chance constraint reformulation
        @constraint(model,
            dot(z, μ̂Y) - θ * t1 ≥ w - Φinv * t2
        )

        # SOC constraints
        @constraint(model, [t1, z[1], z[2], dot(z, A)] in SecondOrderCone())
        @constraint(model, [t2; L2 * z] in SecondOrderCone())

        # Objective
        @objective(model, Min, dot(c, z))

        optimize!(model)

        
        if termination_status(model) == MOI.OPTIMAL
            opt_value = objective_value(model)

            # Out of sample satisfaction
            ẑ = value.(z)
            n_samples = size(out_of_sample_data, 2)
            count = 0

            for i in 1:n_samples
                if dot(out_of_sample_data[:, i], ẑ) ≥ w
                    count += 1
                end
            end

            satisfaction = count / n_samples
        else
            opt_value = nothing 
            satisfaction = nothing
        end
        
        push!(results, (ε, θ, opt_value, satisfaction, contextual_opt_value, contextual_satisfaction))
    end
end

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3 Max
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 7 columns and 11 nonzeros
Model fingerprint: 0xce107965
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 9 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 6.000e+00
 Factor Ops : 1.400e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.02383718e+01  1.02383718e+01  2.22e+00 4.87

In [6]:
results

Row,epsilon,theta,opt_value,satisfaction,contextual_opt_value,contextual_satisfaction
,Float64,Float64,Any,Any,Any,Any
1,0.05,0.01,325.56,0.9623,154.814,0.9456
2,0.05,0.05,435.375,0.9663,174.171,0.9499
3,0.05,0.1,751.685,0.9713,206.421,0.9561
4,0.05,0.2,,,327.727,0.9661
5,0.07,0.01,170.725,0.9461,109.548,0.9241
6,0.07,0.05,196.659,0.9512,118.884,0.9297
7,0.07,0.1,242.647,0.9566,133.055,0.9379
8,0.07,0.2,454.861,0.9673,174.69,0.9508
9,0.09,0.01,123.679,0.9288,88.7766,0.9061


In [7]:
# latex_table = pretty_table(results; backend = Val(:latex))
io = IOBuffer()

pretty_table(
    io,
    results;
    backend = Val(:latex),
    formatters = ft_printf("%.2f", 1:ncol(results))  # 所有列保留两位小数
)

latex_code = String(take!(io))
println(latex_code)

\begin{tabular}{rrrrrr}
  \hline
  \textbf{epsilon} & \textbf{theta} & \textbf{opt\_value} & \textbf{satisfaction} & \textbf{contextual\_opt\_value} & \textbf{contextual\_satisfaction} \\
  \texttt{Float64} & \texttt{Float64} & \texttt{Any} & \texttt{Any} & \texttt{Any} & \texttt{Any} \\\hline
  0.05 & 0.01 & 325.56 & 0.96 & 154.81 & 0.95 \\
  0.05 & 0.05 & 435.38 & 0.97 & 174.17 & 0.95 \\
  0.05 & 0.10 & 751.69 & 0.97 & 206.42 & 0.96 \\
  0.05 & 0.20 & nothing & nothing & 327.73 & 0.97 \\
  0.07 & 0.01 & 170.73 & 0.95 & 109.55 & 0.92 \\
  0.07 & 0.05 & 196.66 & 0.95 & 118.88 & 0.93 \\
  0.07 & 0.10 & 242.65 & 0.96 & 133.06 & 0.94 \\
  0.07 & 0.20 & 454.86 & 0.97 & 174.69 & 0.95 \\
  0.09 & 0.01 & 123.68 & 0.93 & 88.78 & 0.91 \\
  0.09 & 0.05 & 136.70 & 0.94 & 94.80 & 0.91 \\
  0.09 & 0.10 & 157.38 & 0.94 & 103.60 & 0.92 \\
  0.09 & 0.20 & 225.45 & 0.96 & 127.18 & 0.94 \\
  0.10 & 0.01 & 110.33 & 0.92 & 81.95 & 0.89 \\
  0.10 & 0.05 & 120.57 & 0.93 & 87.06 & 0.90 \\
  0.10 & 0.10 & 136